# Dataset CRUD Testing

Testing the CRUD operations and dataset import functionality.

**Prerequisites:** Run `make setup` to start PostgreSQL and run migrations.

In [3]:
import polars as pl
from pathlib import Path

from bookdb.db.session import SessionLocal
from bookdb.datasets.crud import BookCRUD, AuthorCRUD, UserCRUD, BookListCRUD
from bookdb.datasets.processor import import_dataset, preview_dataset, read_file, import_authors, import_books
from bookdb.datasets.repository import Repository

# session = SessionLocal()

## Import GoodReads data with repo class

In [ ]:
repository = Repository()

PROJECT_ROOT = Path.cwd().parent.parent
DATA_DIR = PROJECT_ROOT / "data"

# Import authors dataset
authors_file_path = DATA_DIR / "raw_goodreads_book_authors.parquet"
author_stats = repository.import_authors(authors_file_path)
print(author_stats)

# Import books dataset
book_file_path = DATA_DIR / "raw_goodreads_book_works.parquet"
book_stats = repository.import_books(book_file_path)
print(book_stats)

## Import GoodReads data to DBs

In [7]:
# Preview authors dataset
PROJECT_ROOT = Path.cwd().parent.parent
DATA_DIR = PROJECT_ROOT / "data"

authors_file_path = DATA_DIR / "raw_goodreads_book_authors.parquet"

preview = preview_dataset(authors_file_path)
for item in preview:
    print(item)
    print(f"{item['name']}")

{'average_rating': '3.98', 'author_id': '604031', 'text_reviews_count': '7', 'name': 'Ronald J. Fields', 'ratings_count': '49'}
Ronald J. Fields
{'average_rating': '4.08', 'author_id': '626222', 'text_reviews_count': '28716', 'name': 'Anita Diamant', 'ratings_count': '546796'}
Anita Diamant
{'average_rating': '3.92', 'author_id': '10333', 'text_reviews_count': '5075', 'name': 'Barbara Hambly', 'ratings_count': '122118'}
Barbara Hambly
{'average_rating': '3.68', 'author_id': '9212', 'text_reviews_count': '36262', 'name': 'Jennifer Weiner', 'ratings_count': '888522'}
Jennifer Weiner
{'average_rating': '3.82', 'author_id': '149918', 'text_reviews_count': '96', 'name': 'Nigel Pennick', 'ratings_count': '1740'}
Nigel Pennick


In [8]:
# Import authors dataset
author_stats = import_authors(authors_file_path)
print(author_stats)

{'rows': 829529, 'authors_created': 825736, 'authors_skipped': 3788, 'errors': 5}


In [9]:
# Preview books dataset
PROJECT_ROOT = Path.cwd().parent.parent
DATA_DIR = PROJECT_ROOT / "data"

book_file_path = DATA_DIR / "raw_goodreads_book_works.parquet"

preview = preview_dataset(book_file_path)
for item in preview:
    print(item)
    print(f"{item['original_title']}")

{'books_count': '1', 'reviews_count': '6', 'original_publication_month': '8', 'default_description_language_code': '', 'text_reviews_count': '1', 'best_book_id': '5333265', 'original_publication_year': '1984', 'original_title': 'W. C. Fields: A Life on Film', 'rating_dist': '5:1|4:1|3:1|2:0|1:0|total:3', 'default_chaptering_book_id': '', 'original_publication_day': '', 'original_language_id': '', 'ratings_count': '3', 'media_type': 'book', 'ratings_sum': '12', 'work_id': '5400751'}
W. C. Fields: A Life on Film
{'books_count': '22', 'reviews_count': '10162', 'original_publication_month': '', 'default_description_language_code': '', 'text_reviews_count': '741', 'best_book_id': '25717', 'original_publication_year': '2001', 'original_title': 'Good Harbor', 'rating_dist': '5:517|4:1787|3:2763|2:966|1:196|total:6229', 'default_chaptering_book_id': '', 'original_publication_day': '', 'original_language_id': '', 'ratings_count': '6229', 'media_type': 'book', 'ratings_sum': '20150', 'work_id': 

In [10]:
# Import books dataset
book_stats = import_books(book_file_path)
print(book_stats)

{'rows': 1521962, 'books_created': 609703, 'books_skipped': 37203, 'errors': 875056}


## Test CRUD Operations

In [ ]:
session = SessionLocal()

# Get an author
author = AuthorCRUD.get_or_create(session, "Ronald J. Fields")
print(f"Author: {author.name} (ID: {author.id})")

# Create a book with author
book = BookCRUD.create_with_authors(
    session,
    author_names=["Ronald J. Fields"],
    title="W.C. Fields by Himself",
    pages_number=510,
    publish_year=1973,
)
print(f"Book: {book.title} by {[a.name for a in book.authors]}")

session.commit()

Author: George Orwell (ID: 1)
Book: Animal Farm by ['George Orwell']


In [ ]:
# Search for books
results = BookCRUD.search_by_title(session, "W.C. Fields")
print(f"Found {len(results)} books:")
for b in results:
    print(f"  - {b.title} ({b.publish_year})")

Found 3 books:
  - Animal Farm (1945)
  - Animal Farm (1945)
  - Animal Farm (1945)


In [5]:
# Create user and book list
user = UserCRUD.get_or_create(session, "reader@example.com", "Book Reader")
book_list = BookListCRUD.create(session, user.id, "Classics")
BookListCRUD.add_book(session, book_list.id, book.id)

print(f"User: {user.name}")
print(f"List '{book_list.name}' has {len(book_list.books)} book(s)")

session.commit()

User: Book Reader
List 'Classics' has 1 book(s)


## Test csv support

In [ ]:
# Create a sample CSV
sample = pl.DataFrame({
    "title": ["The Great Gatsby", "1984", "Pride and Prejudice", "Brave New World"],
    "authors": ["F. Scott Fitzgerald", "George Orwell", "Jane Austen", "Aldous Huxley"],
    "pages_number": [180, 328, 279, 268],
    "publisher_name": ["Scribner", "Secker & Warburg", "T. Egerton", "Chatto & Windus"],
    "publish_year": [1925, 1949, 1813, 1932],
})

sample_path = Path("/tmp/sample_books.csv")
sample.write_csv(sample_path)
print(f"Created: {sample_path}")

Created: /tmp/sample_books.csv


title,authors,pages_number,publisher_name,publish_year
str,str,i64,str,i64
"""The Great Gatsby""","""F. Scott Fitzgerald""",180,"""Scribner""",1925
"""1984""","""George Orwell""",328,"""Secker & Warburg""",1949
"""Pride and Prejudice""","""Jane Austen""",279,"""T. Egerton""",1813
"""Brave New World""","""Aldous Huxley""",268,"""Chatto & Windus""",1932


In [7]:
# Preview the dataset before importing
preview = preview_dataset(sample_path, n=3)
for item in preview:
    print(f"Book: {item['book']['title']}")
    print(f"  Authors: {item['authors']}")
    print(f"  Year: {item['book']['publish_year']}\n")

Book: The Great Gatsby
  Authors: ['F. Scott Fitzgerald']
  Year: 1925

Book: 1984
  Authors: ['George Orwell']
  Year: 1949

Book: Pride and Prejudice
  Authors: ['Jane Austen']
  Year: 1813



In [8]:
# Import the dataset
stats = import_dataset(sample_path)
print("Import stats:")
for k, v in stats.items():
    print(f"  {k}: {v}")

Import stats:
  rows: 4
  books_created: 4
  books_skipped: 0
  authors_createderrors: 0


In [9]:
# Verify the imported books
session = SessionLocal()

gatsby = BookCRUD.get_by_title(session, "The Great Gatsby")
if gatsby:
    b = gatsby[0]
    print(f"{b.title}")
    print(f"  Authors: {[a.name for a in b.authors]}")
    print(f"  Published: {b.publish_year}")
    print(f"  Pages: {b.pages_number}")

The Great Gatsby
  Authors: ['F. Scott Fitzgerald']
  Published: 1925
  Pages: 180


## Cleanup

In [ ]:
session.close()

Session closed
